In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pyglstudy as gl
from pyglstudy.group_lasso import *
import numpy as np

In [3]:
n = 1000
p = 100000
n_groups = int(p/100)
alpha = 1.0
seed = 0

np.random.seed(seed)
X, beta, y, groups, group_sizes = generate_group_lasso_data(n, p, n_groups).values()

In [8]:
X_tilde = np.copy(X)

In [9]:
out = gl.transform_data(X_tilde, groups, group_sizes, 16)

In [10]:
# compute cost-per-action:
2.4 / (n * np.sum(np.minimum(n, group_sizes) * group_sizes)) * 16

1.8441394784158842e-09

In [11]:
np.linalg.norm(X_tilde, axis=0)**2, out['A_diag']

(array([1.21188665e+04, 9.33702302e-01, 9.06024963e-01, ...,
        3.51062727e-01, 3.29342882e-01, 3.06440714e-01]),
 array([1.21188665e+04, 9.33702302e-01, 9.06024963e-01, ...,
        3.51062727e-01, 3.29342882e-01, 3.06440714e-01]))